In [1]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


In [77]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)

genre_gn_all = pd.read_json('genre_gn_all.json', typ = 'series') # 장르 읽어오기

train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)
# 장르코드 : gnr_code, 장르명 : gnr_name
genre_df = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})    



In [78]:
display(train_df.head(1))
display(song_df.head(1))
# print(genre_df.columns)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0


# 전처리
- train_df, song_df의 `song_id`를 가지고 merge할 것임.
- songs와 genre와 plylst로 진행해볼 것

## 사용할 컬럼만 고르기

In [87]:
train_df.head(1)

,tags,plylst_title,songs,like_cnt,updt_date,ply_id
0,[락],여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,61281


In [86]:
train_df['ply_id'] = train_df['id']
train_df.drop('id', axis=1, inplace=True)
train_df.drop(['updt_date','tags'], axis=1, inplace=True)
train_df.head(1)

,tags,plylst_title,songs,like_cnt,updt_date,ply_id
0,[락],여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,61281


## train_df의 `songs`랑 song_df `id`랑 같은걸까? 

In [73]:
temp_song = song_df[['song_name','id']]
print(temp_song[temp_song['id'] == 562083])
# 맞다고 가정

        song_name      id
562083  Honeymoon  562083


## train_df, song_df를 songs를 기준으로 merge하자

In [76]:
song_df['song_id'] = song_df['id']
display(song_df.head(1))
print(song_df.columns)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_id


Index(['song_gn_dtl_gnr_basket', 'issue_date', 'album_name', 'album_id',
       'artist_id_basket', 'song_name', 'song_gn_gnr_basket',
       'artist_name_basket', 'id', 'song_id'],
      dtype='object')


In [69]:
train_song_df = pd.concat([train_df, song_df], axis=1)
display(train_song_df.head(1))
print(train_song_df.columns)
train_song_df = train_song_df[['']]

,tags,id,plylst_title,songs,like_cnt,updt_date,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_id
0,[락],61281.0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71.0,2013-12-19 18:36:19.000,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,0


Index(['tags', 'id', 'plylst_title', 'songs', 'like_cnt', 'updt_date',
       'song_gn_dtl_gnr_basket', 'issue_date', 'album_name', 'album_id',
       'artist_id_basket', 'song_name', 'song_gn_gnr_basket',
       'artist_name_basket', 'id', 'song_id'],
      dtype='object')


KeyError: "None of [Index([''], dtype='object')] are in the [columns]"